In [3]:
# import libaries
import geopandas as gpd
import pandas as pd
import json
import numpy as np
from sqlalchemy import create_engine
import time
from datetime import date, timedelta
from rl_v2g import CarsharingEnv
import math
import stable_baselines3
from stable_baselines3 import PPO
from gym.utils.env_checker import check_env as checker_gym
from stable_baselines3.common.env_checker import check_env as checker_baselines3
import torch
import torch.nn as nn
# load the database credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

# create connection string
connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


C:\Users\domin\anaconda3\envs\simulation_car_sharing\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
import IPython

In [5]:
%load_ext snakeviz

# Application of Car-sharing Simulation Environment 

Choose the timespan for the simulation. The simulation will be executed chronologically, starting from the first day (2019-1-1) and continuing for subsequent days (2019-1-2, 2019-1-3, etc.). If a start date other than 2019-1-1 is selected, the "Start simulation" cell below may need to be modified.

In [6]:
# set learn period
start_date = date(2019, 1, 8)
#end_date = date(2020, 7, 27)
end_date = date(2019, 1, 10)
start_week = 1

# set simulation period
start_date_simulation = date(2019, 1, 1)
end_date_simulation = date(2019, 1, 8)
start_week_simulation = 0

# calculate number of days to learn
nr_iterations = (end_date - start_date).days

# calculate number of days to simulate
nr_iterations_simulation = (end_date_simulation - start_date_simulation).days

# Load data for simulation

### Car-sharing stations

In [7]:
# get station geodata, create spatial index
sql = " SELECT * FROM msc_2023_dominik.distinct_stations"
stations = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
stations.sindex
stations.head()

,station_no,geom
0,2901,POINT (2555501.836 1145060.068)
1,2905,POINT (2752963.411 1260089.916)
2,2910,POINT (2501877.645 1126218.900)
3,2913,POINT (2682234.096 1243208.370)
4,2918,POINT (2736874.744 1253090.505)


### Vehicle information

In [8]:
# get vehicle data
sql = "SELECT * FROM msc_2023_dominik.vehicle_information ORDER BY vehicle_no"
vehicles = pd.read_sql(sql, engine)
vehicles.head()

,index,vehicle_category,vehicle_no,model_name,brand_name,charge_power,battery_capacity,range
0,2962,Minivan,106516,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
1,2963,Minivan,106517,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
2,2964,Minivan,106518,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
3,2965,Minivan,106519,eVito 129KB Tourer Pro 3200,Mercedes-Benz,11.0,100.0,378.0
4,2966,Combi,106526,Enyaq iV80,Skoda,11.0,82.0,420.0


### Reservations

In [9]:
# get daily reservations, save in dict for fast data access
delta = timedelta(days=1)
reservations_dict = {}
start_date_reservations = start_date
while start_date_reservations <= end_date:
    sql = """SELECT reservation_no, start_station_no, vehicle_no, reservationfrom_time_discrete, drive_firststart_time_discrete, 
            drive_lastend_time_discrete, reservation_duration, revenue_distance, required_soc, revenue_duration, drive_km, 
            (floor(EXTRACT(epoch FROM (date_trunc('hour', TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) + 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes' 
                                - date_trunc('hour', TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) - 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes'
                               )) / 900) * 900 + 900) / 900 AS drive_duration
            FROM msc_2023_dominik.reservations_long_time 
            WHERE  DATE(reservationfrom_discrete_date) = '{}' or  DATE(drive_firststart_discrete_date) = '{}' 
            ORDER BY reservationfrom_discrete""".format(start_date_reservations, start_date_reservations)
    reservations = pd.read_sql(sql, engine)
    reservations_dict[start_date_reservations.strftime('%Y-%m-%d')] = reservations
    start_date_reservations += delta
reservations_dict[(start_date).strftime('%Y-%m-%d')].head()

,reservation_no,start_station_no,vehicle_no,reservationfrom_time_discrete,drive_firststart_time_discrete,drive_lastend_time_discrete,reservation_duration,revenue_distance,required_soc,revenue_duration,drive_km,drive_duration
0,24281754,4714,114582,72.0,716.0,716.0,1804.0,0.0,0.000000,0.0,1.0,1.0
1,24281745,4714,114572,72.0,715.0,715.0,1804.0,0.0,0.000000,0.0,1.0,1.0
2,22919882,4173,116242,576.0,734.0,925.0,672.0,0.0,9.500000,0.0,19.0,191.0
3,24154068,3057,113667,604.0,706.0,924.0,424.0,68.5,51.500000,265.0,103.0,219.0
4,24340023,2902,116881,604.0,701.0,982.0,380.0,178.4,53.095238,301.5,223.0,283.0


In [10]:
reservations_dict.keys()

dict_keys(['2019-01-08', '2019-01-09', '2019-01-10'])

In [11]:
# get daily reservations, save in dict for fast data access
delta = timedelta(days=1)
reservations_dict_simulation = {}
start_date_reservations = start_date_simulation
while start_date_reservations <= end_date_simulation:
    sql = """SELECT reservation_no, start_station_no, vehicle_no, reservationfrom_time_discrete, drive_firststart_time_discrete, 
            drive_lastend_time_discrete, reservation_duration, revenue_distance, required_soc, revenue_duration, drive_km, 
            (floor(EXTRACT(epoch FROM (date_trunc('hour', TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) + 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes' 
                                - date_trunc('hour', TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) - 
                                floor(EXTRACT(minute FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS')) / 15) * interval '15 minutes'
                               )) / 900) * 900 + 900) / 900 AS drive_duration
            FROM msc_2023_dominik.reservations_long_time 
            WHERE  DATE(reservationfrom_discrete_date) = '{}' or  DATE(drive_firststart_discrete_date) = '{}' 
            ORDER BY reservationfrom_discrete""".format(start_date_reservations, start_date_reservations)
    reservations = pd.read_sql(sql, engine)
    reservations_dict_simulation[start_date_reservations.strftime('%Y-%m-%d')] = reservations
    start_date_reservations += delta
reservations_dict_simulation[(start_date_simulation).strftime('%Y-%m-%d')].head()

,reservation_no,start_station_no,vehicle_no,reservationfrom_time_discrete,drive_firststart_time_discrete,drive_lastend_time_discrete,reservation_duration,revenue_distance,required_soc,revenue_duration,drive_km,drive_duration
0,24519192,2886,114874,0.0,0.0,4.0,6.0,10.50,5.384615,5.25,14.0,4.0
1,24519174,1557,115969,0.0,0.0,49.0,50.0,45.75,30.500000,43.75,61.0,50.0
2,24514447,2702,114871,0.0,4.0,5.0,6.0,2.66,1.538462,7.50,4.0,2.0
3,24519221,3165,116525,0.0,0.0,96.0,96.0,0.00,0.000000,0.00,0.0,97.0
4,24519097,4407,113833,0.0,2.0,9.0,10.0,13.00,10.000000,6.25,20.0,7.0


In [12]:
reservations_dict_simulation.keys()

dict_keys(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08'])

### Electicity prices for charging

In [13]:
# get charging costs data
prices = ""
for i in range(0, 480, 5):
    price = i / 20
    prices += '"Price_chf_kwh_{}", '.format(price)

sql = """SELECT {} "Delivery day" FROM msc_2023_dominik.charging_costs WHERE "Delivery day" >=  '{}' and "Delivery day" <=  '{}' ORDER BY "Delivery day" """.format(prices, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

charging_costs = pd.read_sql(sql, engine)
charging_costs.head()

,Price_chf_kwh_0.0,Price_chf_kwh_0.25,Price_chf_kwh_0.5,Price_chf_kwh_0.75,Price_chf_kwh_1.0,Price_chf_kwh_1.25,Price_chf_kwh_1.5,Price_chf_kwh_1.75,Price_chf_kwh_2.0,Price_chf_kwh_2.25,...,Price_chf_kwh_21.75,Price_chf_kwh_22.0,Price_chf_kwh_22.25,Price_chf_kwh_22.5,Price_chf_kwh_22.75,Price_chf_kwh_23.0,Price_chf_kwh_23.25,Price_chf_kwh_23.5,Price_chf_kwh_23.75,Delivery day
0,0.055044,0.055044,0.055044,0.055044,0.049861,0.049861,0.049861,0.049861,0.049106,0.049106,...,0.064710,0.064667,0.064667,0.064667,0.064667,0.060604,0.060604,0.060604,0.060604,2019-01-08
1,0.061434,0.061434,0.061434,0.061434,0.057124,0.057124,0.057124,0.057124,0.056649,0.056649,...,0.067070,0.067124,0.067124,0.067124,0.067124,0.063622,0.063622,0.063622,0.063622,2019-01-09
2,0.062458,0.062458,0.062458,0.062458,0.057996,0.057996,0.057996,0.057996,0.057705,0.057705,...,0.072285,0.069775,0.069775,0.069775,0.069775,0.061369,0.061369,0.061369,0.061369,2019-01-10


In [14]:
# save in dict for fast data access
delta = timedelta(days=1)
charging_costs_dict = {}
start_date_electricity = start_date
while start_date_electricity <= end_date:
    electricity_price_day = charging_costs[charging_costs["Delivery day"].dt.date == start_date_electricity].drop(["Delivery day"],axis = 1).iloc[0].values
    charging_costs_dict[start_date_electricity.strftime('%Y-%m-%d')] = electricity_price_day
    start_date_electricity += delta

In [15]:
charging_costs_dict.keys()

dict_keys(['2019-01-08', '2019-01-09', '2019-01-10'])

In [16]:
# get charging costs data
prices = ""
for i in range(0, 480, 5):
    price = i / 20
    prices += '"Price_chf_kwh_{}", '.format(price)

sql = """SELECT {} "Delivery day" FROM msc_2023_dominik.charging_costs WHERE "Delivery day" >=  '{}' and "Delivery day" <=  '{}' ORDER BY "Delivery day" """.format(prices, start_date_simulation.strftime('%Y-%m-%d'), end_date_simulation.strftime('%Y-%m-%d'))

charging_costs = pd.read_sql(sql, engine)
charging_costs.head()

,Price_chf_kwh_0.0,Price_chf_kwh_0.25,Price_chf_kwh_0.5,Price_chf_kwh_0.75,Price_chf_kwh_1.0,Price_chf_kwh_1.25,Price_chf_kwh_1.5,Price_chf_kwh_1.75,Price_chf_kwh_2.0,Price_chf_kwh_2.25,...,Price_chf_kwh_21.75,Price_chf_kwh_22.0,Price_chf_kwh_22.25,Price_chf_kwh_22.5,Price_chf_kwh_22.75,Price_chf_kwh_23.0,Price_chf_kwh_23.25,Price_chf_kwh_23.5,Price_chf_kwh_23.75,Delivery day
0,0.054160,0.054160,0.054160,0.054160,0.052522,0.052522,0.052522,0.052522,0.050906,0.050906,...,0.054634,0.059397,0.059397,0.059397,0.059397,0.059387,0.059387,0.059387,0.059387,2019-01-01
1,0.053848,0.053848,0.053848,0.053848,0.052436,0.052436,0.052436,0.052436,0.044785,0.044785,...,0.065669,0.064731,0.064731,0.064731,0.064731,0.063546,0.063546,0.063546,0.063546,2019-01-02
2,0.057145,0.057145,0.057145,0.057145,0.054419,0.054419,0.054419,0.054419,0.052070,0.052070,...,0.067910,0.065755,0.065755,0.065755,0.065755,0.061800,0.061800,0.061800,0.061800,2019-01-03
3,0.056671,0.056671,0.056671,0.056671,0.055701,0.055701,0.055701,0.055701,0.053826,0.053826,...,0.069678,0.069764,0.069764,0.069764,0.069764,0.065410,0.065410,0.065410,0.065410,2019-01-04
4,0.067953,0.067953,0.067953,0.067953,0.063018,0.063018,0.063018,0.063018,0.058977,0.058977,...,0.064548,0.065874,0.065874,0.065874,0.065874,0.065863,0.065863,0.065863,0.065863,2019-01-05


In [17]:
# save in dict for fast data access
delta = timedelta(days=1)
charging_costs_dict_simulation = {}
start_date_electricity = start_date_simulation
while start_date_electricity <= end_date_simulation:
    electricity_price_day = charging_costs[charging_costs["Delivery day"].dt.date == start_date_electricity].drop(["Delivery day"],axis = 1).iloc[0].values
    charging_costs_dict_simulation[start_date_electricity.strftime('%Y-%m-%d')] = electricity_price_day
    start_date_electricity += delta

In [18]:
charging_costs_dict_simulation.keys()

dict_keys(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08'])

### Secondary energy prices (for V2G)

In [19]:
# get v2g price data
sql = """SELECT "Timestamp", "Secondary_positive_v2g_prices_chf_kwh", "Secondary_negative_v2g_prices_chf_kwh" FROM msc_2023_dominik.v2g_prices WHERE "Timestamp" >=  '{}' and "Timestamp" <=  '{}' ORDER BY "Timestamp" """.format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
v2g_prices = pd.read_sql(sql, engine)
v2g_prices.head()

,Timestamp,Secondary_positive_v2g_prices_chf_kwh,Secondary_negative_v2g_prices_chf_kwh
0,2019-01-08 00:00:00,0.052801,0.035201
1,2019-01-08 00:15:00,0.052687,0.035132
2,2019-01-08 00:30:00,0.052687,0.035132
3,2019-01-08 00:45:00,0.052687,0.035132
4,2019-01-08 01:00:00,0.052687,0.035132


In [20]:
# save in dict for fast data access
delta = timedelta(days=1)
v2g_price_dict = {}
start_date_v2g = start_date
while start_date_v2g <= end_date:
    v2g_price_day_positive = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_positive_v2g_prices_chf_kwh"].values
    v2g_price_day_negative = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_negative_v2g_prices_chf_kwh"].values
    v2g_price_dict[start_date_v2g.strftime('%Y-%m-%d')] = [v2g_price_day_positive, v2g_price_day_negative]
    start_date_v2g += delta

In [21]:
v2g_price_dict.keys()

dict_keys(['2019-01-08', '2019-01-09', '2019-01-10'])

In [22]:
# get v2g price data
sql = """SELECT "Timestamp", "Secondary_positive_v2g_prices_chf_kwh", "Secondary_negative_v2g_prices_chf_kwh" FROM msc_2023_dominik.v2g_prices WHERE "Timestamp" >=  '{}' and "Timestamp" <=  '{}' ORDER BY "Timestamp" """.format(start_date_simulation.strftime('%Y-%m-%d'), end_date_simulation.strftime('%Y-%m-%d'))
v2g_prices = pd.read_sql(sql, engine)
v2g_prices.head()

,Timestamp,Secondary_positive_v2g_prices_chf_kwh,Secondary_negative_v2g_prices_chf_kwh
0,2019-01-01 00:00:00,0.068370,0.045588
1,2019-01-01 00:15:00,0.068838,0.045896
2,2019-01-01 00:30:00,0.068838,0.045896
3,2019-01-01 00:45:00,0.068838,0.045896
4,2019-01-01 01:00:00,0.068838,0.045896


In [23]:
# save in dict for fast data access
delta = timedelta(days=1)
v2g_price_dict_simulation = {}
start_date_v2g = start_date_simulation
while start_date_v2g <= end_date_simulation:
    v2g_price_day_positive = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_positive_v2g_prices_chf_kwh"].values
    v2g_price_day_negative = v2g_prices[v2g_prices['Timestamp'].dt.date == pd.Timestamp(start_date_v2g).date()].drop(["Timestamp"],axis = 1)["Secondary_negative_v2g_prices_chf_kwh"].values
    v2g_price_dict_simulation[start_date_v2g.strftime('%Y-%m-%d')] = [v2g_price_day_positive, v2g_price_day_negative]
    start_date_v2g += delta

In [24]:
v2g_price_dict_simulation.keys()

dict_keys(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08'])

# Check environment

In [23]:
# check if enviornment fullfils requirements of gym and stable-baselines3

# load discrete table
sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(0)
data = pd.read_sql(sql, engine)
    
# load discrete planned reservation table
sql =  "SELECT * FROM msc_2023_dominik.planned_reservations_discrete_{} ORDER BY vehicle_no".format(0)
planned_reservations = pd.read_sql(sql, engine)
    
# load discrete planned reservation duration table
sql =  "SELECT * FROM msc_2023_dominik.planned_durations_discrete_{} ORDER BY vehicle_no".format(0)
planned_durations = pd.read_sql(sql, engine)
end = time.time()

# get number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577
    
count = 0
# iterate over weeks (for loading weekly discrete data)
for week_nr in range(start_week, 1):
    # iteration for each day
    for day in range(98,99,96):
        
        # calculate number of timesteps since first day of simulation
        timesteps_since_start = count * 96
        
        # all requested days are simulated
        if count == nr_iterations:
            break
            
        # get date
        date_day = pd.to_datetime(data.columns[day-97]).date()
        date_day_string = date_day.strftime('%Y-%m-%d')
        
        # load reservations
        reservations = reservations_dict[date_day_string]
        
        # load electricity prices for charging
        electricity_price = charging_costs_dict[date_day_string]
        
        # load secondary energy prices for v2g
        v2g_price = v2g_price_dict[date_day_string]
    
        # select discrete data of day
        daily_data = data.iloc[:,day-97:day-1]
        planned_reservations_day = planned_reservations.iloc[:,day-97 + 1:day + 1] 
        planned_durations_day = planned_durations.iloc[:,day-97 + 1:day + 1] 
        
        # create environment
        env = CarsharingEnv(stations, vehicles, planned_bookings = True, v2g_penalty = 10000, penalty_per_kwh = 0, daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day)
        
        # check implementation 
        checker_gym(env)
        checker_baselines3(env)
        # count number of simulated days
        count += 1
        



# Train Agent

In [25]:
# check support of GPU
stable_baselines3.common.utils.get_device(device='cpu')

device(type='cpu')

Start simulation by running the following cell:

In [34]:
def validation(model):
    nr_vehicles = len(vehicles)
    
    global nr_iterations_simulation

    # maximal simulation length
    if nr_iterations_simulation > 577:
        nr_iterations_simulation = 577
        
    total_reward = 0

    count = 0
    # iterate over weeks (for loading weekly discrete data)
    for week_nr in range(start_week_simulation, math.ceil((start_week_simulation * 7 + nr_iterations_simulation) / 7)):
        # load discrete car-sharing table
        sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(week_nr)
        data = pd.read_sql(sql, engine)

        # load discrete planned reservation table
        sql =  "SELECT * FROM msc_2023_dominik.planned_reservations_discrete_{} ORDER BY vehicle_no".format(week_nr)
        planned_reservations = pd.read_sql(sql, engine)

        # load discrete planned reservation duration table
        sql =  "SELECT * FROM msc_2023_dominik.planned_durations_discrete_{} ORDER BY vehicle_no".format(week_nr)
        planned_durations = pd.read_sql(sql, engine)
        

        # iteration for each day
        for day in range(98,676,96):

            # calculate number of timesteps since first day of simulation
            timesteps_since_start = count * 96

            # measure calculation time of episode
            start = time.time()

            # all requested days are simulated
            if count == nr_iterations:
                break

            # get date
            date_day = pd.to_datetime(data.columns[day-97]).date()
            date_day_string = date_day.strftime('%Y-%m-%d')

            # load reservations
            reservations = reservations_dict_simulation[date_day_string]

            # load electricity prices for charging
            electricity_price = charging_costs_dict_simulation[date_day_string]

            # load secondary energy prices for v2g
            v2g_price = v2g_price_dict_simulation[date_day_string]

            # select discrete data of day
            daily_data = data.iloc[:,day-97:day-1]
            planned_reservations_day = planned_reservations.iloc[:,day-97 + 1:day + 1] 
            planned_durations_day = planned_durations.iloc[:,day-97 + 1:day + 1] 

            # reset environment at beginnning of simulation
            if count == 0:
                environment = CarsharingEnv(stations, vehicles, planned_bookings = True, 
                                   daily_data = daily_data, reservations = reservations, electricity_price = electricity_price,
                                    timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day, 
                                    planned_durations = planned_durations_day, random_seed_number = 42, v2g_penalty = 10000)
                s = environment.reset()

            # beginn new day without reseting environemnt 
            else:
                environment.next_day(daily_data, reservations, electricity_price, timesteps_since_start, v2g_price, planned_reservations_day, planned_durations_day)

            # simulate day in 15 min steps
            done = False
            counter = 0
            while not done:

                # get your action 
                act, _states = model.predict(s)

                # proceed one time step
                s, rew, done, _ = environment.step(act)
                
                total_reward += rew

                counter +=1

            # measure time needed for simulation of day
            end = time.time()
            print("")
            print("Total Episode Time: ", end-start)
            print("")

            # plot summary statistics of episode (day)
            #environment.daily_summary_statistics()

            # plot summary statistic over full simulation period
            #if count == nr_iterations - 1:
              #  environment.episode_summary_statistics(nr_iterations)

            # count number of simulated days
            count += 1
            
    return total_reward

In [35]:
#%%snakeviz
# get number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577

reward_list = []
count_list = []
count = 0
# iterate over weeks (for loading weekly discrete data)
for week_nr in range(start_week, math.ceil((start_week * 7 + nr_iterations) / 7)):
    # load discrete car-sharing table
    sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(week_nr)
    data = pd.read_sql(sql, engine)
    print(week_nr)
    
    # load discrete planned reservation table
    sql =  "SELECT * FROM msc_2023_dominik.planned_reservations_discrete_{} ORDER BY vehicle_no".format(week_nr)
    planned_reservations = pd.read_sql(sql, engine)
    
    # load discrete planned reservation duration table
    sql =  "SELECT * FROM msc_2023_dominik.planned_durations_discrete_{} ORDER BY vehicle_no".format(week_nr)
    planned_durations = pd.read_sql(sql, engine)
        
    # iteration for each day
    for day in range(98,676,96):
        
        # calculate number of timesteps since first day of simulation
        timesteps_since_start = count * 96 + week_nr*7*96
        
        # measure calculation time of episode
        start = time.time()
        
        # all requested days are simulated
        if count == nr_iterations:
            break
            
        # get date
        date_day = pd.to_datetime(data.columns[day-97]).date()
        date_day_string = date_day.strftime('%Y-%m-%d')
        
        # load reservations
        reservations = reservations_dict[date_day_string]
        
        # load electricity prices for charging
        electricity_price = charging_costs_dict[date_day_string]
        
        # load secondary energy prices for v2g
        v2g_price = v2g_price_dict[date_day_string]
    
        # select discrete data of day
        daily_data = data.iloc[:,day-97:day-1]
        planned_reservations_day = planned_reservations.iloc[:,day-97 + 1:day + 1] 
        planned_durations_day = planned_durations.iloc[:,day-97 + 1:day + 1] 
        
        # reset environment at beginnning of simulation
        if count == 0:
            
            # create environment
            env = CarsharingEnv(stations, vehicles, planned_bookings = True, v2g_penalty = 10000, penalty_per_kwh = 1.0, daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day, max_distance_car_assingment=300)
            
            # create RL model
            model = PPO("MlpPolicy",env, verbose=1, n_steps=95, batch_size = 95,device ="cpu")
            model.policy.mlp_extractor.policy_net = nn.Sequential(
                nn.Linear(model.policy.mlp_extractor.policy_net[0].in_features, 1024),  # Modify the first layer
                nn.Tanh(),
                nn.Linear(1024, 5120),
                model.policy.mlp_extractor.policy_net[3]
            )
            model.policy.action_net =  nn.Linear(5120, 13260)
            model.policy.mlp_extractor.value_net = nn.Sequential(
                nn.Linear(model.policy.mlp_extractor.value_net[0].in_features, 1028),  # Modify the first layer
                nn.Tanh(),
                nn.Linear(1028, 128),
                model.policy.mlp_extractor.value_net[3]
            )
            model.policy.value_net =  nn.Linear(128, 1)
            
            r = validation(model)
            print("Validation reward: ", r, " CHF   Episodes learned: ",count)
            reward_list.append(r)
            count_list.append(count)

            
            #model.policy.to(device)
            
        else: 
            env.load_new_data(daily_data = daily_data, reservations = reservations,
                           electricity_price = electricity_price, timesteps_since_start = timesteps_since_start, v2g_price = v2g_price, planned_reservations = planned_reservations_day,
                           planned_durations = planned_durations_day)
            s = env.reset()
            
        
        # learn one episode
        model.learn(total_timesteps=95, reset_num_timesteps=False)
            
          
        end = time.time()
        print("")
        print("Total Episode Time: ", end-start," " ,count)
        print("")
        
        if count in range(5,nr_iterations + 1,10):
            r = validation(model)
            print("Validation reward: ", r, " CHF   Episodes learned: ",count)
            reward_list.append(r)
            count_list.append(count)

            
        count += 1

1
Reset environment to timestamp:  672
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Reset environment to timestamp:  0
Reset environment to timestamp:  0

Total Episode Time:  33.61339783668518


Total Episode Time:  34.509225368499756

Validation reward:  -15291.662614217757  CHF   Episodes learned:  0
Reset environment to timestamp:  672
Reset environment to timestamp:  672
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95       |
|    ep_rew_mean     | -5.7e+04 |
| time/              |          |
|    fps             | 1        |
|    iterations      | 1        |
|    time_elapsed    | 50       |
|    total_timesteps | 95       |
---------------------------------

Total Episode Time:  150.62600803375244   0

Reset environment to timestamp:  768
Reset environment to timestamp:  768
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95       |
|    ep_re

In [31]:
print(reward_list)

[-40686.723115463785, -40279.86276328254]


In [88]:
model.policy.mlp_extractor.policy_net = nn.Sequential(
    nn.Linear(model.policy.mlp_extractor.policy_net[0].in_features, 1024),  # Modify the first layer
    nn.Tanh(),
    nn.Linear(1024, 5120),
    model.policy.mlp_extractor.policy_net[3]
)
model.policy.action_net =  nn.Linear(5120, 13260)
model.policy.mlp_extractor.value_net = nn.Sequential(
    nn.Linear(model.policy.mlp_extractor.value_net[0].in_features, 1028),  # Modify the first layer
    nn.Tanh(),
    nn.Linear(1028, 128),
    model.policy.mlp_extractor.value_net[3]
)
model.policy.value_net =  nn.Linear(128, 1)


# print network
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=17682, out_features=1024, bias=True)
      (1): Tanh()
      (2): Linear(in_features=1024, out_features=5120, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=17682, out_features=1028, bias=True)
      (1): Tanh()
      (2): Linear(in_features=1028, out_features=128, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=5120, out_features=13260, bias=True)
  (value_net): Linear(in_features=128, out_features=1, bias=True)
)